# Morningstar mutual fund dataset analysis with spark sql

In [3]:
!pip install findspark

In [4]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

In [10]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [5]:
import json

In [6]:
! ls -lh *MS*

-rw-rw-r-- 1 29999 29999  87M Nov 26  2020 MS-MF.csv
-rw-r--r-- 1 root  root  1.8M Jun  4 09:29 MS-MFsample.csv

MSMF:
total 89M
-rw-rw-r-- 1 29999 29999  87M Nov 26  2020 MS-MF.csv
-rw-r--r-- 1 29999 29999 1.8M Jun  4 09:29 MS-MFsample.csv


In [13]:
! cat MS-MF.csv | wc -l 

57604


In [10]:
! head -n 1 MS-MF.csv 

### Create DF

In [181]:
df = se.read.csv('MS-MF.csv',header= True)
df.printSchema()


root
 |-- ticker: string (nullable = true)
 |-- isin: string (nullable = true)
 |-- fund_name: string (nullable = true)
 |-- inception_date: string (nullable = true)
 |-- category: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- analyst_rating: string (nullable = true)
 |-- risk_rating: string (nullable = true)
 |-- performance_rating: string (nullable = true)
 |-- investment_strategy: string (nullable = true)
 |-- investment_managers: string (nullable = true)
 |-- dividend_frequency: string (nullable = true)
 |-- fund_benchmark: string (nullable = true)
 |-- morningstar_benchmark: string (nullable = true)
 |-- equity_style: string (nullable = true)
 |-- equity_style_score: string (nullable = true)
 |-- equity_size: string (nullable = true)
 |-- equity_size_score: string (nullable = true)
 |-- price_prospective_earnings: string (nullable = true)
 |-- price_book_ratio: string (nullable = true)
 |-- price_sales_ratio: string (nullable = true)
 |-- price_cash_flow_rati

In [23]:
df.registerTempTable("sample")

In [24]:
se.sql("""
select 
    ticker,fund_trailing_return_ytd
from sample
""").show(5)

+----------+
|    ticker|
+----------+
|0P00000BOW|
|0P00000ESH|
|0P00000ESL|
|0P00000FI7|
|0P00000FIA|
+----------+
only showing top 5 rows



#### Some queries to find 'Russian stocks' funds 

In [55]:
'fund_name'

se.sql("""
select  
   fund_name, FLOAT(fund_trailing_return_ytd) as ytd, FLOAT(fund_trailing_return_3years) as y3,
    FLOAT(fund_trailing_return_5years) as y5
from sample
where 
lower(fund_name) LIKE "%russia%"
order by ytd desc

""").show()

+--------------------+-----+-----+-----+
|           fund_name|  ytd|   y3|   y5|
+--------------------+-----+-----+-----+
|Schroder Internat...|15.05|11.15|19.16|
|Schroder Internat...|14.43| 10.2|18.43|
|Schroder Internat...|14.15|10.65|18.32|
|Schroder Internat...| 14.1| 9.72|17.78|
|Schroder Internat...|13.78| 9.79|17.69|
|Schroder Internat...|13.69| null| null|
|Schroder Internat...|13.49| 9.63|17.71|
|Schroder Internat...|13.33| 9.26|17.11|
|Schroder Internat...|13.27| 9.56|17.66|
|Schroder Internat...|13.01| 8.57|16.54|
|Schroder Internat...| 12.8| 9.31|17.43|
|Schroder Internat...|12.71| 8.64|16.45|
|Schroder Internat...|12.69| 9.29|17.42|
|Schroder Internat...|12.21| 8.65|16.72|
|Schroder Internat...| 12.1| 8.64|16.72|
|Aberdeen Standard...|-2.92| 3.61|11.94|
|Liontrust Russia ...|-2.96|10.42| 17.6|
|UBS (Lux) Equity ...|-3.02| 9.42|17.88|
|UBS (Lux) Equity ...|-3.34| 9.02|17.46|
|Aberdeen Standard...|-4.15|  3.2|12.18|
+--------------------+-----+-----+-----+
only showing top

In [54]:
se.sql("""
select  
    fund_name, FLOAT(fund_trailing_return_ytd) as ytd, FLOAT(fund_trailing_return_3years) as y3,
    FLOAT(fund_trailing_return_5years) as y5
from sample
where 
lower(fund_name) LIKE "%russia%"
order by y3 desc

""").show()

+--------------------+-----+-----+-----+
|           fund_name|  ytd|   y3|   y5|
+--------------------+-----+-----+-----+
|Schroder Internat...|15.05|11.15|19.16|
|Schroder Internat...|14.15|10.65|18.32|
|Liontrust Russia ...|-2.96|10.42| 17.6|
|Schroder Internat...|14.43| 10.2|18.43|
|Schroder Internat...|13.78| 9.79|17.69|
|Schroder Internat...| 14.1| 9.72|17.78|
|Schroder Internat...|13.49| 9.63|17.71|
|Schroder Internat...|13.27| 9.56|17.66|
|UBS (Lux) Equity ...|-3.02| 9.42|17.88|
|Schroder Internat...| 12.8| 9.31|17.43|
|Schroder Internat...|12.69| 9.29|17.42|
|Schroder Internat...|13.33| 9.26|17.11|
|UBS (Lux) Equity ...|-3.34| 9.02|17.46|
|SEB Russia Fund C...|-9.06| 8.77|19.12|
|Schroder Internat...|12.21| 8.65|16.72|
|Schroder Internat...| 12.1| 8.64|16.72|
|Schroder Internat...|12.71| 8.64|16.45|
|Schroder Internat...|13.01| 8.57|16.54|
|Pictet-Russian Eq...|-5.27| 8.48|20.18|
|Liontrust Russia ...|-5.07| 8.35|16.11|
+--------------------+-----+-----+-----+
only showing top

In [53]:
se.sql("""
select  
    fund_name, FLOAT(fund_trailing_return_ytd) as ytd, FLOAT(fund_trailing_return_3years) as y3,
    FLOAT(fund_trailing_return_5years) as y5
from sample
where 
lower(fund_name) LIKE "%russia%"

order by y5 desc

""").show()

+--------------------+------+-----+-----+
|           fund_name|   ytd|   y3|   y5|
+--------------------+------+-----+-----+
|Pictet-Russian Eq...| -5.27| 8.48|20.18|
|Pictet-Russian Eq...| -5.84|  7.7|19.25|
|Pictet-Russian Eq...|  -5.9| 7.65|19.22|
|Schroder Internat...| 15.05|11.15|19.16|
|SEB Russia Fund C...| -9.06| 8.77|19.12|
|Schroder Internat...| 14.43| 10.2|18.43|
|Pictet-Russian Eq...|  -6.4| 6.97|18.42|
|Pictet-Russian Eq...| -6.45| 6.93|18.39|
|Pictet-Russian Eq...| -6.44| 6.93|18.34|
|Pictet-Russian Eq...| -6.71| 6.91|18.33|
|Schroder Internat...| 14.15|10.65|18.32|
|SEB Russia Fund C...|-10.08| 8.11|17.97|
|Pictet-Russian Eq...| -5.88| 7.61|17.89|
|UBS (Lux) Equity ...| -3.02| 9.42|17.88|
|Schroder Internat...|  14.1| 9.72|17.78|
|Schroder Internat...| 13.49| 9.63|17.71|
|Schroder Internat...| 13.78| 9.79|17.69|
|Schroder Internat...| 13.27| 9.56|17.66|
|Liontrust Russia ...| -2.96|10.42| 17.6|
|East Capital Russ...|  -7.5| 5.74| 17.5|
+--------------------+------+-----

In [76]:
se.sql("""
select  
    substr(fund_name, 0, 15) , avg(FLOAT(fund_trailing_return_ytd)) as ytd, 
    avg(FLOAT(fund_trailing_return_3years)) as y3,
    avg(FLOAT(fund_trailing_return_5years)) as y5,
    count(*)
from sample
where 
lower(fund_name) LIKE "%russia%"
group by substr(fund_name, 0, 15) 
order by y5 desc

""").show()

+------------------------+-------------------+------------------+------------------+--------+
|substr(fund_name, 0, 15)|                ytd|                y3|                y5|count(1)|
+------------------------+-------------------+------------------+------------------+--------+
|         SEB Russia Fund| -9.570000171661377| 8.440000057220459|18.545000076293945|       2|
|         Pictet-Russian |-6.7191667556762695| 6.851818084716797| 18.47299995422363|      12|
|         Schroder Intern|  13.38733336130778| 9.504285744258336| 17.51000009264265|      15|
|         UBS (Lux) Equit| -4.130000034968059|  8.18333355585734| 16.99999936421712|       3|
|         Liontrust Russi| -4.015000104904175| 9.385000228881836|16.855000495910645|       3|
|         East Capital Ru| -8.051999950408936| 4.983999967575073|16.846000099182127|       5|
|         DWS Russia LC E| -8.460000038146973| 5.940000057220459|16.100000381469727|       1|
|         HSBC Global Inv|-12.973555543687608| 5.06210523843

In [107]:
se.sql("""
select  
     category, count(*),first(fund_name)  -- distinct(equity_style)
from sample
where 
lower(fund_name) LIKE "%russia%" 
    group by category 
--order by y5 desc


""").show(truncate = True)

+--------------------+--------+--------------------+
|            category|count(1)|    first(fund_name)|
+--------------------+--------+--------------------+
|       Russia Equity|     107|HSBC Global Inves...|
|Emerging Europe e...|       1|SEB Eastern Europ...|
|         BRIC Equity|      15|Schroder Internat...|
+--------------------+--------+--------------------+



In [139]:
se.sql("""
select  
    substr(fund_name, 0, 15), avg(FLOAT(fund_trailing_return_ytd)) as ytd, 
    avg(FLOAT(fund_trailing_return_3years)) as y3,
    avg(FLOAT(fund_trailing_return_5years)) as y5 
    --first(equity_style)
    --count(*)
from sample
where 
category = 'Russia Equity' 
--and 
--equity_style == 'Blend'
group by substr(fund_name, 0, 15)
order by y5 desc
limit 5
""").show(truncate =True)

+------------------------+-------------------+-----------------+------------------+
|substr(fund_name, 0, 15)|                ytd|               y3|                y5|
+------------------------+-------------------+-----------------+------------------+
|         SEB Russia Fund| -9.570000171661377|8.440000057220459|18.545000076293945|
|         Pictet-Russian |-6.7191667556762695|6.851818084716797| 18.47299995422363|
|         UBS (Lux) Equit| -4.130000034968059| 8.18333355585734| 16.99999936421712|
|         Liontrust Russi| -4.015000104904175|9.385000228881836|16.855000495910645|
|         East Capital Ru| -8.051999950408936|4.983999967575073|16.846000099182127|
+------------------------+-------------------+-----------------+------------------+



###  Leaders in trailing_return_5years и trailing_return_3years

In [150]:
se.sql("""
(select  
    substr(fund_name, 0, 15)
from sample
where 
category = 'Russia Equity' 
--and 
--equity_style == 'Blend'
group by substr(fund_name, 0, 15)
order by avg(FLOAT(fund_trailing_return_5years)) desc 
limit 10)

INTERSECT 
(select  
    substr(fund_name, 0, 15)
from sample
where 
category = 'Russia Equity' 
group by substr(fund_name, 0, 15)
order by avg(FLOAT(fund_trailing_return_3years)) desc
limit 10)

--INTERSECT 

--(select  
--    substr(fund_name, 0, 15)
--from sample
--where 
--category = 'Russia Equity' 
--group by substr(fund_name, 0, 15)
--order by avg(FLOAT(fund_trailing_return_ytd)) desc
--limit 10)
""").show(truncate =True)

+------------------------+
|substr(fund_name, 0, 15)|
+------------------------+
|         SEB Russia Fund|
|         Pictet-Russian |
|         UBS (Lux) Equit|
|         Liontrust Russi|
|         DWS Russia LC E|
|         HSBC Global Inv|
|         BNP Paribas Fun|
|         Raiffeisen-Russ|
+------------------------+



###  Get exact fund names

In [169]:
se.sql("""
select  
    fund_name, (FLOAT(fund_trailing_return_ytd)) as ytd, 
    (FLOAT(fund_trailing_return_3years)) as y3,
    (FLOAT(fund_trailing_return_5years)) as y5,
    analyst_rating
    --first(equity_style)
    --count(*)
from sample
where 
category = 'Russia Equity' 
and 
fund_name LIKE 'SEB Strategy Gr%' --'DWS Russia LC E%' --'UBS (Lux) Equit%' --'Pictet-Russian%' --'SEB Russia Fund%'
--and analyst_rating  'null'
--and 
--equity_style == 'Blend'
--group by substr(fund_name, 0, 15)
order by fund_name desc

""").show(100,truncate =False)

+--------------------------------+----+----+-----+--------------+
|fund_name                       |ytd |y3  |y5   |analyst_rating|
+--------------------------------+----+----+-----+--------------+
|SEB Strategy Growth Fund C SEK  |9.96|4.05|9.33 |null          |
|SEB Strategy Growth Fund C H-EUR|7.54|5.13|11.28|null          |
+--------------------------------+----+----+-----+--------------+



### Get highest return by category

#### TODO: Need to clean data

In [176]:
se.sql("""
select  
    -- fund_name,top5_holdings
    avg(FLOAT(fund_trailing_return_ytd)) as ytd, 
    avg(FLOAT(fund_trailing_return_3years)) as y3,
    avg(FLOAT(fund_trailing_return_5years)) as y5,
    count(*) as cnt,
    (category)
from sample
--where 
--lower(fund_name) LIKE "%russia%"
group by category 
--order by ytd desc
""").show(truncate=True)

+-------------------+--------------------+------------------+----+--------------------+
|                ytd|                  y3|                y5| cnt|            category|
+-------------------+--------------------+------------------+----+--------------------+
|  901.9908411051347|   3.376120096055348|355781.51437501056| 550|    Other Allocation|
| 19.741260116603918|   6.619185345315672|14.724336458890924| 639|Asia ex-Japan Equity|
|  5.875937485054601|  2.3925581602163093| 12.87090084144661| 163|Japan Small/Mid-C...|
| 10.572857311793737|  2.5579999446868897| 5.063333352406819|   7|SEK Moderate Allo...|
| 0.9340946517095216|  3.7507809574272417| 3.146539161497479|1031| Alt - Multistrategy|
| 3.9093301458079277|3.5270577254649684E7|2.7159842425209333| 222|Global Bond - GBP...|
| 7.4748705203478165|   7.257314501800196|11.756969692078904|1936|Global Large-Cap ...|
| 18.489230857445644|  0.7607999968528748| 7.520833344509204|  26|        Korea Equity|
|   4.70034471976346|  1.5313043

#### Get most popular morningstar_benchmark

In [184]:
se.sql("""
select 
    (morningstar_benchmark), count(*) as cnt
from sample
group by morningstar_benchmark 
order by cnt desc
""").show(truncate =False) #.show(5)


+----------------------------------------+-----+
|morningstar_benchmark                   |cnt  |
+----------------------------------------+-----+
|null                                    |16219|
|MSCI ACWI NR USD                        |2204 |
|MSCI EM NR USD                          |1899 |
|Morningstar UK Mod Tgt Alloc NR GBP     |1612 |
|FTSE AllSh TR GBP                       |1577 |
|MSCI ACWI Growth NR USD                 |1278 |
|JPM EMBI Global Diversified TR USD      |1002 |
|Morningstar UK Mod Adv Tgt Alloc NR GBP |935  |
|TOPIX TR JPY                            |898  |
|Russell 1000 TR USD                     |841  |
|BBgBarc Global Aggregate TR USD         |840  |
|BBgBarc Global Aggregate TR Hdg EUR     |782  |
|MSCI Europe NR EUR                      |779  |
|Morningstar EU Mod Gbl Tgt Alloc NR EUR |768  |
|MSCI World High Dividend Yield NR USD   |746  |
|Cat 50%Barclays US Agg TR&50%FTSE Wld TR|717  |
|JPM GBI-EM Global Diversified TR USD    |693  |
|BBgBarc Global Aggr

# Get highest return

##### TODO: Need to clean data

In [180]:
se.sql("""
select  
    fund_name, top5_holdings,
    (FLOAT(fund_trailing_return_ytd)) as ytd, 
    (FLOAT(fund_trailing_return_3years)) as y3,
    (FLOAT(fund_trailing_return_5years)) as y5
    --count(*) as cnt,
    --(category)
from sample
--where 
--lower(fund_name) LIKE "%russia%"
--group by category 
order by y5 desc

""").show(100,truncate=True)

+--------------------+--------------------+------------+-----+------------+
|           fund_name|       top5_holdings|         ytd|   y3|          y5|
+--------------------+--------------------+------------+-----+------------+
|Epsilon Fund - Eu...|                 EUR| 6.6311002E8| null| 4.2099799E9|
|Vanguard Emerging...|                 GBP|       261.0| null|2.66349005E9|
|Pareto Nordic Cro...|                 NOK|    2.6198E8| null|1.85238003E9|
|Pareto Nordic Cro...|                 EUR|         0.0| null|1.85238003E9|
|Vanguard Emerging...|                 EUR|      161.96| null|1.57367002E9|
|AB - Internationa...|                 USD|    2.3279E8| null|1.49905997E9|
|AB - Internationa...|                 USD|1.00321997E9| null|1.49905997E9|
|AB - Internationa...|                 USD|1.00321997E9| null|1.49905997E9|
|AB - Internationa...|                 USD|    2.1786E8| null|1.46547994E9|
|AB - Internationa...|                 GBP|    260000.0| null|1.45712998E9|
|AB - Intern

# Stop Spark

In [37]:
sc.stop()